In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [49]:
def dataset_options():
    # choosing datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    
    return data

In [54]:
def load_data(): 
    x,y=[],[]
    c,c1=0,0   
    # feature to extract
    mfcc = True
    chroma=True
    mel=False
    
    data = dataset_options()
    #paths = []
    if data['ravdess']:
        for file in glob.glob("C:\\Users\\Lavanya Ravinutala\\Downloads\\ravdess\\Actor_*\\*.wav"):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]] #to get emotion from emotions dictionary using key
            
            if emotion not in observed_emotions:
                continue
            feature=extract_feature(file, mfcc,chroma,mel)
            c=c+1
            x.append(feature)
            y.append(emotion)
    if data['tess']:
        for file in glob.glob("C:\\Users\\Lavanya Ravinutala\\Downloads\\TESS\\*AF_*\\*.wav"):
            file_name=os.path.basename(file)
            emotion=file_name.split("_")[2][:-4] #split and remove .wav
            

            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions:
                continue

            feature=extract_feature(file, mfcc,chroma,mel)
            c1=c1+1
            #print(feature)
            x.append(feature)
            y.append(emotion)
   
    print("total files in ravdess dataset:",c)
    print("total files in tess dataset:",c1) 
    return {"X":x,"y":y}

In [55]:
import time
st= time.time()
Trial_dict = load_data()

total files in ravdess dataset: 768
total files in tess dataset: 0


In [56]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [57]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [58]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\Lavanya Ravinutala\\Downloads\\ravdess\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [59]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [60]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [61]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [62]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [63]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [64]:
y_pred=model.predict(x_test)

In [65]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.19%
